In [ ]:
import openai
from flask import Flask, request, jsonify

app = Flask(__name__)

# OpenAI API 키 설정
openai.api_key = 'APIKEY'

@app.route('/chatbot', methods=['POST'])
def chatbot():
    data = request.json
    question = data.get('question', '')

    try:
        # GPT-3를 사용하여 질문에 대한 답변 생성
        response = openai.Completion.create(
          engine="text-davinci-003", 
          prompt=question,
          temperature=0.7,
          max_tokens=150,
          top_p=1.0,
          frequency_penalty=0.0,
          presence_penalty=0.0
        )
        answer = response.choices[0].text.strip()
    except Exception as e:
        answer = f"죄송합니다, 오류가 발생했습니다: {str(e)}"

    return jsonify({"answer": answer})

if __name__ == '__main__':
    app.run(debug=True, port=5000)


In [ ]:
import openai

openai.api_key = 'APIKEY'

# 파인튜닝 파일 업로드
response = openai.File.create(file=open("finetune_data.jsonl"), purpose='fine-tune')
file_id = response['id']

# 파인튜닝 작업 제출
finetune_response = openai.FineTune.create(
  training_file=file_id,
  model="gpt-3.5-turbo",
  n_epochs=1,
  learning_rate_multiplier=0.1,
  prompt_loss_weight=0.1,
)

print(f"파인튜닝 작업이 시작되었습니다. 작업 ID: {finetune_response['id']}")


In [ ]:
finetune_id = finetune_response['id']
finetune_status = openai.FineTune.retrieve(id=finetune_id)
print(f"파인튜닝 작업 상태: {finetune_status['status']}")


In [ ]:
import openai
from flask import Flask, request, jsonify

app = Flask(__name__)

# OpenAI API 키 설정
openai.api_key = 'APIKEY'

# 파인튜닝된 모델의 ID 설정
finetuned_model_id = "FINETUNEDMODELID"

@app.route('/chatbot', methods=['POST'])
def chatbot():
    data = request.json
    question = data.get('question', '')

    try:
        # 파인튜닝된 모델을 사용하여 질문에 대한 답변 생성
        response = openai.Completion.create(
          model=finetuned_model_id, # 파인튜닝된 모델의 ID 사용
          prompt=question,
          temperature=0.7,
          max_tokens=150,
          top_p=1.0,
          frequency_penalty=0.0,
          presence_penalty=0.0
        )
        answer = response.choices[0].text.strip()
    except Exception as e:
        answer = f"죄송합니다, 오류가 발생했습니다: {str(e)}"

    return jsonify({"answer": answer})

if __name__ == '__main__':
    app.run(debug=True, port=5000)


In [ ]:
curl -X POST http://localhost:5000/chatbot -H "Content-Type: application/json" -d "{\"question\":\"지금 날씨 어때?\"}"
